In [1]:
!pip install -q duckduckgo-search langchain_experimental


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Build Search Tool using DuckDuckGoSearch

from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

In [3]:
search_tool = DuckDuckGoSearchRun()

search_tool.invoke('top news in india today')

"Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at NDTV.com. Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. LIVE news updates: Pahalgam attack was act of economic warfare, says EAM Jaishankar Today's top updates: S Jaishankar will travel to Washington DC to participate in the Quad Foreign Ministers' Meeting on Tuesday. Catch all the news developments from around the world here Get Top News of the day with latest top news headlines of today. Get updated with current top news stories from India and the world only on Zee News. Get all the latest news, live updates and content about India from across the BBC."

In [4]:
search_tool.get_graph().print_ascii()

        +----------+         
        | DDGInput |         
        +----------+         
              *              
              *              
              *              
    +-------------------+    
    | duckduckgo_search |    
    +-------------------+    
              *              
              *              
              *              
+--------------------------+ 
| duckduckgo_search_output | 
+--------------------------+ 


In [5]:
print(search_tool.name)
print(search_tool.args)
print(search_tool.description)

duckduckgo_search
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.


# Built-in Tool - Shell Tool

In [6]:
from langchain_community.tools import ShellTool

In [7]:
shell_tool = ShellTool()    # this tool is use to run command 


In [8]:
result = shell_tool.invoke("dir") # it showing the all directory 
result

Executing command:
 dir


f:\study\LangGraph_LangChain\llmenv\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


' Volume in drive F has no label.\r\n Volume Serial Number is CCA1-D187\r\n\r\n Directory of f:\\study\\LangGraph_LangChain\\LangChain\\12_Tools\r\n\r\n07/02/2025  02:11 PM    <DIR>          .\r\n07/02/2025  10:49 AM    <DIR>          ..\r\n06/27/2025  04:20 PM         2,788,678 00_Basic.ipynb\r\n07/02/2025  02:14 PM            15,479 01_Tool_Exp.ipynb\r\n07/02/2025  02:11 PM               930 02_Tool_Calling.ipynb\r\n               3 File(s)      2,805,087 bytes\r\n               2 Dir(s)  285,483,634,688 bytes free\r\n'

In [9]:
result

' Volume in drive F has no label.\r\n Volume Serial Number is CCA1-D187\r\n\r\n Directory of f:\\study\\LangGraph_LangChain\\LangChain\\12_Tools\r\n\r\n07/02/2025  02:11 PM    <DIR>          .\r\n07/02/2025  10:49 AM    <DIR>          ..\r\n06/27/2025  04:20 PM         2,788,678 00_Basic.ipynb\r\n07/02/2025  02:14 PM            15,479 01_Tool_Exp.ipynb\r\n07/02/2025  02:11 PM               930 02_Tool_Calling.ipynb\r\n               3 File(s)      2,805,087 bytes\r\n               2 Dir(s)  285,483,634,688 bytes free\r\n'

# Create Custom Tools

In [10]:
# step 1 : create a function

from langchain_core.tools import tool

@tool
def multiply(a:int,b:int)->int:
    """Multiply two numbers"""
    return a*b

In [11]:
multiply.invoke({"a":1,"b":10})

10

In [12]:
print(multiply.name)
print(multiply.args)
print(multiply.description)

multiply
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
Multiply two numbers


# Method 2 - Using StructuredTool

In [13]:
from langchain.tools import StructuredTool
from pydantic import BaseModel,Field


In [14]:
class MultiplyInput(BaseModel):
    a:int=Field(...,description="Enter interger value of a")
    b:int=Field(...,description="Enter interger value of b")  # The ... is a shorthand for saying “no default value, this field is required”.

In [15]:
def multiply(a:int,b:int)->int:
    return a*b

In [16]:
from langchain.tools import StructuredTool

In [17]:
multiply_tool = StructuredTool.from_function(func=multiply,
                                             name="mulitplyfunction",
                                             description="it use to multiple two function",
                                             args_schema=MultiplyInput)



In [18]:
multiply_tool.invoke({"a":1,"b":10})

10

In [19]:
import os
from langchain_groq import ChatGroq
from langchain.schema import Document

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="llama3-70b-8192")


In [26]:
messages = []

In [27]:
bound_model = model.bind_tools([multiply_tool])

# important 

`content is empty string always remember model never execut the tool it suggest suggest which tool may use to perform task`

In [28]:
from langchain_core.messages import HumanMessage, AIMessage
query = HumanMessage(content="what is multiplication of 30,20")

messages.append(query)

tool_result= bound_model.invoke(messages) # content is empty string always remember model never execut the tool it suggest suggest which tool may use to perform task

tool_result


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'yt16bbcms', 'function': {'arguments': '{"a":30,"b":20}', 'name': 'mulitplyfunction'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 929, 'total_tokens': 968, 'completion_time': 0.140522494, 'prompt_time': 0.037055716, 'queue_time': 0.05797213899999999, 'total_time': 0.17757821}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--000aaa13-2b13-499c-b8ca-99a0144ffa7b-0', tool_calls=[{'name': 'mulitplyfunction', 'args': {'a': 30, 'b': 20}, 'id': 'yt16bbcms', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 39, 'total_tokens': 968})

In [29]:
import json

def extract_tool(tool_result):
    # print(tool_result)

    
    messages.append(tool_result)
    all_tools = tool_result.tool_calls
    for tool in all_tools:
        if tool['name']=="mulitplyfunction":
            result = multiply_tool.invoke(tool)
            messages.append(result)
            # message_list.append(AIMessage(content=json.dumps(result)))
    return messages

final_result = extract_tool(tool_result)
final_result

[HumanMessage(content='what is multiplication of 30,20', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'yt16bbcms', 'function': {'arguments': '{"a":30,"b":20}', 'name': 'mulitplyfunction'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 929, 'total_tokens': 968, 'completion_time': 0.140522494, 'prompt_time': 0.037055716, 'queue_time': 0.05797213899999999, 'total_time': 0.17757821}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--000aaa13-2b13-499c-b8ca-99a0144ffa7b-0', tool_calls=[{'name': 'mulitplyfunction', 'args': {'a': 30, 'b': 20}, 'id': 'yt16bbcms', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 39, 'total_tokens': 968}),
 ToolMessage(content='600', name='mulitplyfunction', tool_call_id='yt16bbcms')]

In [30]:
messages

[HumanMessage(content='what is multiplication of 30,20', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'yt16bbcms', 'function': {'arguments': '{"a":30,"b":20}', 'name': 'mulitplyfunction'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 929, 'total_tokens': 968, 'completion_time': 0.140522494, 'prompt_time': 0.037055716, 'queue_time': 0.05797213899999999, 'total_time': 0.17757821}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--000aaa13-2b13-499c-b8ca-99a0144ffa7b-0', tool_calls=[{'name': 'mulitplyfunction', 'args': {'a': 30, 'b': 20}, 'id': 'yt16bbcms', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 39, 'total_tokens': 968}),
 ToolMessage(content='600', name='mulitplyfunction', tool_call_id='yt16bbcms')]

In [32]:
bound_model.invoke(messages).content

'The result of multiplying 30 and 20 is 600.'

# ToolKit

In [42]:
# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [43]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [44]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
